In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('movielens/movie_rating.csv')
ratings

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0
5,Mick LaSalle,Lady in the Water,3.0
6,Mick LaSalle,Snakes on a Plane,4.0
7,Mick LaSalle,Just My Luck,2.0
8,Mick LaSalle,Superman Returns,3.0
9,Mick LaSalle,You Me and Dupree,2.0


In [3]:
ratings_pivot = ratings.pivot_table(columns='critic', index='title', values='rating')
ratings_pivot

critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


In [4]:
sim_matrix = ratings_pivot.corr()
sim_matrix

critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
critic,,,,,,
Claudia Puig,1.000000,0.314970,0.028571,0.566947,0.566947,0.893405
Gene Seymour,0.314970,1.000000,0.963796,0.396059,0.411765,0.381246
Jack Matthews,0.028571,0.963796,1.000000,0.747018,0.211289,0.662849
Lisa Rose,0.566947,0.396059,0.747018,1.000000,0.594089,0.991241
Mick LaSalle,0.566947,0.411765,0.211289,0.594089,1.000000,0.924473
Toby,0.893405,0.381246,0.662849,0.991241,0.924473,1.000000


In [8]:
sim_matrix_abs = sim_matrix.abs()

In [5]:
ratings_pivot.loc['Snakes on a Plane']

critic
Claudia Puig     3.5
Gene Seymour     3.5
Jack Matthews    4.0
Lisa Rose        3.5
Mick LaSalle     4.0
Toby             4.5
Name: Snakes on a Plane, dtype: float64

In [10]:
def pred_rating(user, movie):
    if not(movie in ratings_pivot.index):
        return 3.0

    movie_vec = ratings_pivot.loc[movie]
    movie_vec = movie_vec.dropna()
    sim_vec = sim_matrix_abs[user].loc[movie_vec.index]
    weighted_rating = (movie_vec * sim_vec).dropna()

    # if no weighted ratings, use movie_vec to compute rating
    if weighted_rating.size <= 0:
        return movie_vec.mean() if movie_vec.size > 0 else 3.0
    
    # use similarity as confidence to weight the ratings of other users
    return weighted_rating.sum()/sim_vec.sum()

In [14]:
test = pd.DataFrame({'user' : ['Toby'] * 3, 'movie': ['Just My Luck', 'Lady in the Water', 'The Night Listener']})
test

,user,movie
0,Toby,Just My Luck
1,Toby,Lady in the Water
2,Toby,The Night Listener


In [15]:
def predict(estimate_f):
    test_predictions = test.copy(deep=True)
    test_predictions['prediction'] = test_predictions.apply(lambda x: estimate_f(x.user, x.movie), axis=1)

    return test_predictions

In [16]:
predict(pred_rating)

,user,movie,prediction
0,Toby,Just My Luck,2.530981
1,Toby,Lady in the Water,2.832550
2,Toby,The Night Listener,3.347790


In [ ]:
user_mean_ratings = 